In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA, AutoETS, AutoTheta, AutoCES, _TS
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.model_selection import ParameterGrid
from utilsforecast.plotting import plot_series
import pandas as pd
import matplotlib.pyplot as plt
import fn
from toolz.functoolz import pipe
import plotly.graph_objects as go

/home/philip/miniconda3/envs/prophetTest/lib/python3.11/site-packages/statsforecast/core.py:27: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [6]:
# Y_df = AirPassengersDF # Defined in neuralforecast.utils
# Y_df.head()

Y_df = pipe(
  #fn.loadDf("./data/Packets_30days_1minGran-2024-05-06_15_33_52.csv")
  fn.loadDf("./data/TripleWitching-2024-05-06 15_56_54.csv")
  ,fn.keep(["ds","NASDAQ_Canadian_Chix_A"])
  ,lambda df:df.rename(columns={"NASDAQ_Canadian_Chix_A":"y"})
  ,lambda df:df.fillna(0)
  ,fn.addUniqueId("1")
)
Y_df

loadDf: ./data/TripleWitching-2024-05-06 15_56_54.csv


,ds,y,unique_id
0,2024-02-06 14:00:00,34291.2,1
1,2024-02-06 15:00:00,37201.6,1
2,2024-02-06 16:00:00,22200.0,1
3,2024-02-06 17:00:00,26984.0,1
4,2024-02-06 18:00:00,36220.8,1
...,...,...,...
2156,2024-05-06 10:00:00,8.0,1
2157,2024-05-06 11:00:00,589.6,1
2158,2024-05-06 12:00:00,547.2,1
2159,2024-05-06 13:00:00,38060.0,1


In [8]:
season_length = 24 # Define season length as 12 months for monthly data
horizon = 24 # Forecast horizon is set to 1 month
levels = [90,99]
freq = fn.inferFreq(Y_df)
print(freq)
models = [
    AutoARIMA(season_length=season_length), # ARIMA model with automatic order selection and seasonal component
    # AutoETS(season_length=season_length), # ETS model with automatic error, trend, and seasonal component
    # AutoTheta(season_length=season_length), # Theta model with automatic seasonality detection
    # AutoCES(season_length=season_length), # CES model with automatic seasonality detection
]

# Instantiate StatsForecast class with models, data frequency ('M' for monthly),
# and parallel computation on all CPU cores (n_jobs=-1)
sf = StatsForecast(
    models=models, # models for forecasting
    freq=freq,  # frequency of the data
    n_jobs=-1  # number of jobs to run in parallel, -1 means using all processors
)


h


In [16]:
# Generate forecasts for the specified horizon using the sf object
Y_hat_df = sf.forecast(df=Y_df, h=horizon,level=levels) # forecast data
# Display the first few rows of the forecast DataFrame
Y_hat_df.head() # preview of forecasted data
Y_hat_df.reset_index()

KeyboardInterrupt: 

In [17]:
Y_hat_df=Y_hat_df.reset_index()
Y_hat_df.head() # preview of forecasted data


,unique_id,ds,AutoARIMA,AutoARIMA-lo-99,AutoARIMA-lo-90,AutoARIMA-hi-90,AutoARIMA-hi-99
0,1,2024-05-06 15:00:00,10145.863281,-15100.283203,-5975.630371,26267.357422,35392.011719
1,1,2024-05-06 16:00:00,6184.163086,-21329.919922,-11385.572266,23753.898438,33698.246094
2,1,2024-05-06 17:00:00,5999.932617,-22040.580078,-11905.965820,23905.830078,34040.445312
3,1,2024-05-06 18:00:00,6160.269531,-22044.775391,-11850.695312,24171.234375,34365.316406
4,1,2024-05-06 19:00:00,10428.113281,-17858.904297,-7635.196289,28491.421875,38715.128906


In [ ]:
df = Y_hat_insample.reset_index()
df = df.drop(['cutoff','y'],axis=1)
df = df.rename(columns={"NHITS-median": "NHITS"})
df


,unique_id,ds,NHITS,NHITS-lo-99,NHITS-lo-90,NHITS-lo-80,NHITS-hi-80,NHITS-hi-90,NHITS-hi-99
0,a,2024-01-01 12:00:00,-0.108353,-0.407957,-0.338368,-0.150701,0.064993,0.007472,0.273509
1,a,2024-01-01 13:00:00,-0.185011,-0.476949,-0.243333,-0.120072,0.037656,0.044760,-0.063932
2,a,2024-01-01 14:00:00,0.984654,0.255963,0.574604,0.940408,1.537441,1.361575,1.836984
3,a,2024-01-01 15:00:00,0.923275,0.192768,0.694973,0.953472,1.496655,1.410971,1.442286
4,a,2024-01-01 16:00:00,1.289634,0.059408,0.607347,0.950567,1.919309,1.771615,2.333704
5,a,2024-01-01 17:00:00,1.137796,-0.009922,0.658496,1.012793,1.882055,1.745244,2.052947
6,a,2024-01-01 18:00:00,1.132224,-0.196516,0.319200,0.660871,1.584400,1.482992,2.039933
7,a,2024-01-01 19:00:00,0.853178,-0.250263,0.372104,0.713475,1.545412,1.433161,1.727971
8,b,2024-01-01 12:00:00,-0.108353,-0.407957,-0.338368,-0.150701,0.064993,0.007472,0.273509
9,b,2024-01-01 13:00:00,-0.185011,-0.476949,-0.243333,-0.120072,0.037656,0.044760,-0.063932


In [24]:
#p=matplotlib.colors.LinearSegmentedColormap.from_list("", ["red","violet","blue"])
# Y_hat_insample
p = list(["red","blue"])
fig = plot_series(Y_df, Y_hat_df, level=[90], engine="plotly")
# fill='tonexty',  fill='tonexty',

#for r in [1,2]:
for n in ['AutoARIMA-lo-90','AutoARIMA-hi-90']: 
  fig.add_trace(
    go.Scatter(
      x=Y_hat_df['ds'], 
      y=Y_hat_df[n], 
      mode='lines', 
      line=dict(color='lightgrey'), 
      name='99% Prediction Interval'
    ), 
    #row=1, 
    #col=r
  )
 
fig.show()

In [ ]:
sf = StatsForecast(
    df=train, 
    models=models, 
    freq='H', 
    n_jobs=-1
)